In [8]:
import tensorflow as tf
import tensorflow_hub as hub
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.applications.resnet50 import ResNet50

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.20)

train_generator = train_datagen.flow_from_directory(
    r"C:\Users\786me\Desktop\FinalYearProject\ASDScanPath\ASD\Images",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    r"C:\Users\786me\Desktop\FinalYearProject\ASDScanPath\ASD\Images", # same directory as training data
    target_size=(224 , 224),
    batch_size=32,
    class_mode='binary',
    subset='validation') # set as validation data

img_size = (224, 224 , 3)

Found 439 images belonging to 2 classes.
Found 108 images belonging to 2 classes.


In [10]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#Freeze weights 
for layer in base_model.layers[1:170]:
    layer.trainable = False

for i,layer in enumerate(base_model.layers):
    print(i,layer.name,layer.trainable)

0 input_2 True
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_block

In [13]:
model = tf.keras.Sequential([
    base_model,tf.keras.layers.Dropout(0.2)
])
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units= 128, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units= 1, activation = 'sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])


model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 dropout_6 (Dropout)         (None, 7, 7, 2048)        0         
                                                                 
 flatten_3 (Flatten)         (None, 100352)            0         
                                                                 
 dense_6 (Dense)             (None, 128)               12845184  
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 36,433,025
Trainable params: 22,835,457


In [12]:
history = model.fit(train_generator, validation_data=validation_generator, epochs = 8)

Epoch 1/8
14/14 [==============================] - 63s 4s/step - loss: 1.5319 - accuracy: 0.5513 - val_loss: 0.6743 - val_accuracy: 0.6019
Epoch 2/8
14/14 [==============================] - 40s 3s/step - loss: 0.7394 - accuracy: 0.5626 - val_loss: 0.6866 - val_accuracy: 0.6019
Epoch 3/8
14/14 [==============================] - 37s 3s/step - loss: 0.6804 - accuracy: 0.5786 - val_loss: 0.6908 - val_accuracy: 0.6019
Epoch 4/8
14/14 [==============================] - 32s 2s/step - loss: 0.6792 - accuracy: 0.5991 - val_loss: 0.6947 - val_accuracy: 0.3981
Epoch 5/8
14/14 [==============================] - 32s 2s/step - loss: 0.6840 - accuracy: 0.5991 - val_loss: 0.6953 - val_accuracy: 0.3981
Epoch 6/8
14/14 [==============================] - 32s 2s/step - loss: 0.6832 - accuracy: 0.5991 - val_loss: 0.7057 - val_accuracy: 0.3981
Epoch 7/8
14/14 [==============================] - 33s 2s/step - loss: 0.6765 - accuracy: 0.5991 - val_loss: 0.7056 - val_accuracy: 0.3981
Epoch 8/8
14/14 [==========

In [14]:
##Confusion Matrix Generator and ROC Curve
y_pred_real = model.predict(validation_generator)
y_pred = [int(p >= 0.5) for p in y_pred_real] ##With sigmoid 1 neuron
##y_pred = np.argmax(Y_pred, axis=1) ##With softmax and 2 neurons 
y_true = validation_generator.classes
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(validation_generator.classes, y_pred)
print(cm)

4/4 [==============================] - 5s 1s/step
[[65  0]
 [43  0]]
